In [1]:
import pandas as pd
import numpy as np
import statsmodels.tsa.filters.filtertools
# import statsmodels.api as sm
import math
import re
import glob
import os
import sys
module_path = os.path.abspath(os.path.join('../../src/utils'))
if module_path not in sys.path:
    sys.path.append(module_path)
from datetime import datetime

from optimizer import Optimizer
import warnings
warnings.filterwarnings('ignore')

In [2]:
# To display numbers in decimal separated format
pd.options.display.float_format = '{:20,.5f}'.format

# for timing usage: Within 2 mins

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# Part 1. Data Preparation

## 1). configurations: update the parameters at the below cell

In [3]:
## REQUIRED PARAMETERS
# the modeling_stack file path used for the modelling
input_file_path = '../../data/processed_input_data/02_modelling_stack_data_seasonality.csv'
# the 'model_outputs.xlsx' file path from the model_results
final_model_path = '../../models/manual/sales_unit/mix_media_metrics/2021-11-09_model_outputs_v1/model_results_0802/model_outputs.xlsx'
# the 'variables_transformation_summary.xlsx' file path from the model_results
trans_path = '../../models/manual/sales_unit/mix_media_metrics/2021-11-09_model_outputs_v1/model_results_0802/variables_transformation_summary.xlsx'

# Specify whether to use forward optimization
# if forward_opt = False, the stack dataframe from 'input_file_path' will be used for the optimization
# if forward_opt = True, the simulated dataframe will be used
# 1. if you already ran the simulator with the above 'input_file_path', the simulated dataframe will be available as below:
#    e.g. '../../models/manual/new_customer/mix_media_metrics/2021-08-02_model_outputs_v1/simulation_results/Updated_Stack_df.csv'
# 2. do not rename the simulated dataframe 'updated_stack_df.csv', otherwise the system cannot find it.
forward_opt = False

# Select the transformation method, c_curve or s_curve
# transformation_method = 'c_curve'
transformation_method = 's_curve'
# Specify the upper and lower constraint on each of the media variable
# media_range_dict: Dictionary with upper and lower constraint for each media driver

media_range_dict = {'M_ON_DIS_AT_SPEND' : [0.7, 1.3],
                    'M_ON_DIS_CT_ROS_SPEND' : [0.7, 1.3],
                    'M_ON_DIS_HPLO_SPEND' : [0.7, 1.3],
                    'M_ON_DIS_KW_SPEND' : [0.7, 1.3],
                    'M_SEARCH_AUTO_SPEND' : [1.0, 1.0],
                    'M_SEARCH_MAN_SPEND' : [1.0, 1.0],
                    'M_OFF_DIS_WN_SPEND' : [0.7, 1.3],
                    'M_OFF_DIS_PIN_SPEND' : [0.7, 1.3],
                    'M_NATIONAL_TV_SPEND' : [1.0, 1.0],
                   }

# media_range_dict = {}

# optimization start and end dates
# if it is for future time period optimization, the optimization time period should be the same as the simulation time period. 
opt_start_end_date = ('2020-07-01', '2021-01-31')
# modeling start and end dates, RECOMMEND to use modeling start and end dates 
modeling_start_end_date = ('2019-01-01', '2021-04-30')

# Specify the granular components of media (i.e. indivual variables of an aggregated variable)
# granular_media_dict = {'M_ON_DIS_AT_CT_IMP': ['M_ON_DIS_AT_IMP', 'M_ON_DIS_CT_IMP']}
granular_media_dict = {}

perc = 1

# OPTIONAL PARAMETERS

# 1. it will read the total incremental from function model_obj.visualize_ROAS() with the optimization period
# 2. if not available, it will read the total contribution from the simulation result
# 3. it will calculate the inc_contribution automatically if you leave it as None 
# inc_contribution = None 
# inc_contribution = 100742 # new_customer model
inc_contribution = 598913 # unit model

# compute the asp_fpv over the data range used for optimization from stack
# 1. for new customer model: asp_fpv = average of PRICE_NEW_CUSTOMER_OMNI during optimization period
# 2. for omni units model: asp_fpv = average of PRICE during optimization period, (asp_fpv = sales/unit)
# it will calculate the asp_fpv automatically if you leave it as None 
# asp_fpv = None
# asp_fpv = 18.3 # new_customer model
asp_fpv = 6.156 # unit model


# apply_adstock transformation
apply_adstock = True

# pre-process the data stack
df = pd.read_csv(input_file_path)
df['index'] = pd.to_datetime(df['index']) 
df.set_index("index", inplace = True, drop= False)
# Please specify your custom variables that were not part of standard data stack
# Example:
df['M_SEARCH_MAN_CLK']=df['M_SBA_CLK']+df['M_SP_KWB_CLK']
df['M_SEARCH_AUTO_CLK']=df['M_SP_AB_CLK']
df['M_NATIONAL_TV_SPEND']=1000 
df['M_SEARCH_MAN_SPEND']=df['M_SBA_SPEND']+df['M_SP_KWB_SPEND']
df['M_SEARCH_AUTO_SPEND']=df['M_SP_AB_SPEND']
df['RATE_ON_DIS_CT_ROS']=df['M_ON_DIS_CT_ROS_SPEND']/df['M_ON_DIS_CT_ROS_IMP']
df['RATE_SEARCH_AUTO_CLK']=df['M_SEARCH_AUTO_SPEND']/df['M_SEARCH_AUTO_CLK']
df['RATE_SEARCH_MAN_CLK']=df['M_SEARCH_MAN_SPEND']/df['M_SEARCH_MAN_CLK']
df['RATE_NATIONAL_TV']=df['M_NATIONAL_TV_SPEND']/df['M_NATIONAL_TV_IMP']

## 2). parameters preparation: don't need to change anything at the below cell

In [4]:
# read and pass the parameters required for the optimization
input_file_path = input_file_path
final_model_path = final_model_path
trans_path = trans_path

forward_opt = forward_opt
transformation_method = transformation_method

# optimization start and end date
opt_start_end_date = opt_start_end_date

# modeling start and end dates
modeling_start_end_date = modeling_start_end_date

granular_media_dict = granular_media_dict
media_range_dict = media_range_dict
inc_contribution = inc_contribution 
perc = perc
asp_fpv = asp_fpv
apply_adstock = apply_adstock

# Part 2. Optimization

In [5]:
# Initiate the optimization Object
opt_obj = Optimizer(df = df,
                    forward_opt = forward_opt,
                    transformation_method = transformation_method,
                    final_model_path = final_model_path,
                    trans_path = trans_path,
                    opt_start_end_date = opt_start_end_date,
                    modeling_start_end_date = modeling_start_end_date,
                    granular_media_dict = granular_media_dict,
                    media_range_dict = media_range_dict,
                    inc_contribution = inc_contribution,
                    perc = perc,
                    asp_fpv = asp_fpv,
                    apply_adstock = apply_adstock
                   )

Backward optimization is in use!



Variable            Og Spends           Og Contri.  \
0      M_ON_DIS_AT_SPEND        407,082.80480         15,887.34293   
1  M_ON_DIS_CT_ROS_SPEND        106,781.70155            141.40772   
2    M_ON_DIS_HPLO_SPEND         80,448.86500          7,435.20098   
3      M_ON_DIS_KW_SPEND         19,346.25933              0.05296   
4     M_OFF_DIS_WN_SPEND        284,867.35832         13,390.67401   
5    M_OFF_DIS_PIN_SPEND        150,000.00000        112,765.42213   
6    M_NATIONAL_TV_SPEND        215,000.00000        270,150.02037   
7    M_SEARCH_AUTO_SPEND        472,849.46000         61,521.09496   
8     M_SEARCH_MAN_SPEND        213,420.38000         74,172.15731   
9                  Total      1,949,796.82899                  nan   

               Og RoAS               Og Rev  
0              0.24025         97,802.48310  
1              0.00815            870.50593  
2              0.56895         45,771.09726  
3              0.00002              0.32605  
4              0.28937         82,432.98922  
5              4.62789        694,183.93865  
6              7.73509      1,663,043.52542  
7              0.80094        378,723.86057  
8              2.13946        456,603.80040  
9                  nan                  nan

Optimized budget should be between $1,635,238.73 and $2,264,354.93



In [6]:
# Function 1. call the public function to export and display the optimization results
# about global_opt
global_opt = False
# 1. when global_opt = True: optimization method = differential_evolution
# 2. when global_opt = False: optimization method = minimize(method = 'SLSQP')
num_cores = 4

opt_obj.export_and_display_optimization_results(global_opt, num_cores)

Inequality constraints incompatible
Contribution result saved to: /Users/vn51u3a/Desktop/ms_new/Modeling/OLS Model/auto folder_structure/models/manual/sales_unit/mix_media_metrics/2021-11-09_model_outputs_v1/optimization_results/optimization_outputs_v55/contrib_df.csv


Incremental from model  Maximized from Solver           Difference  \
0           555,463.37339          555,463.37339              0.00000   

           %age change  
0              0.00000

Optimization result saved to: /Users/vn51u3a/Desktop/ms_new/Modeling/OLS Model/auto folder_structure/models/manual/sales_unit/mix_media_metrics/2021-11-09_model_outputs_v1/optimization_results/optimization_outputs_v55/df_new.csv

OPTIMIZED BUDGET of $1,949,796.83 is within the lower, upper range of ($1,635,238.73, $2,264,354.93)



Variable            Og Spends           Og Contri.  \
7    M_SEARCH_AUTO_SPEND        472,849.46000         61,521.09496   
0      M_ON_DIS_AT_SPEND        407,082.80480         15,887.34293   
4     M_OFF_DIS_WN_SPEND        284,867.35832         13,390.67401   
6    M_NATIONAL_TV_SPEND        215,000.00000        270,150.02037   
8     M_SEARCH_MAN_SPEND        213,420.38000         74,172.15731   
5    M_OFF_DIS_PIN_SPEND        150,000.00000        112,765.42213   
1  M_ON_DIS_CT_ROS_SPEND        106,781.70155            141.40772   
2    M_ON_DIS_HPLO_SPEND         80,448.86500          7,435.20098   
3      M_ON_DIS_KW_SPEND         19,346.25933              0.05296   
9                  Total      1,949,796.82899                  nan   

               Og RoAS               Og Rev   Recommended Change  \
7              0.80094        378,723.86057              1.00000   
0              0.24025         97,802.48310              1.00000   
4              0.28937         82,432.98922              1.00000   
6              7.73509      1,663,043.52542              1.00000   
8              2.13946        456,603.80040              1.00000   
5              4.62789        694,183.93865              1.00000   
1              0.00815            870.50593              1.00000   
2              0.56895         45,771.09726              1.00000   
3              0.00002              0.32605              1.00000   
9                  nan                  nan                  nan   

              Change %           Opt Spends          Opt Contri.  \
7              0.00000        472,849.46000         61,521.09496   
0              0.00000        407,082.80480         15,887.34293   
4              0.00000        284,867.35832         13,390.67401   
6              0.00000        215,000.00000        270,150.02037   
8              0.00000        213,420.38000         74,172.15731   
5              0.00000        150,000.00000        112,765.42213   
1              0.00000        106,781.70155            141.40772   
2              0.00000         80,448.86500          7,435.20098   
3              0.00000         19,346.25933              0.05296   
9                  nan      1,949,796.82899                  nan   

              Opt RoAS              Opt Rev  
7              0.80094        378,723.86057  
0              0.24025         97,802.48310  
4              0.28937         82,432.98922  
6              7.73509      1,663,043.52542  
8              2.13946        456,603.80040  
5              4.62789        694,183.93865  
1              0.00815            870.50593  
2              0.56895         45,771.09726  
3              0.00002              0.32605  
9                  nan                  nan

Overall Optimization result saved to: /Users/vn51u3a/Desktop/ms_new/Modeling/OLS Model/auto folder_structure/models/manual/sales_unit/mix_media_metrics/2021-11-09_model_outputs_v1/optimization_results/optimization_outputs_v55/roas_df.csv



Actual Sales      Optimized Sales          Actual RoAS  \
0      3,419,432.52659      3,419,432.52659              1.75374   

        Optimized RoAS  
0              1.75374

In [7]:
# Function 2. call the public function to export and display the bound and headroom analysis results
bound_start_end_date = ('2019-10-01', '2020-05-31')
spend = True
bound_analysis = True
headroom_analysis = True
display_zero_for_neg = True

opt_obj.export_and_display_bound_headroom_analysis(bound_start_end_date, spend, headroom_analysis, bound_analysis, display_zero_for_neg)

Headroom Analysis result saved to: /Users/vn51u3a/Desktop/ms_new/Modeling/OLS Model/auto folder_structure/models/manual/sales_unit/mix_media_metrics/2021-11-09_model_outputs_v1/optimization_results/optimization_outputs_v55/headroom_analysis.csv



Variable              Actuals  Head Room for On-Air  \
0      M_ON_DIS_AT_SPEND        388,976.25319         417,640.94267   
1  M_ON_DIS_CT_ROS_SPEND        164,298.32130         207,548.07466   
2    M_ON_DIS_HPLO_SPEND         89,443.26000               0.00000   
3      M_ON_DIS_KW_SPEND         17,985.75000          62,177.94507   
4     M_OFF_DIS_WN_SPEND        153,056.69348         428,188.03366   
5    M_OFF_DIS_PIN_SPEND              0.00000                   nan   
6    M_NATIONAL_TV_SPEND        244,000.00000          93,434.10362   
7    M_SEARCH_AUTO_SPEND        151,072.07891         766,980.26169   
8     M_SEARCH_MAN_SPEND         84,978.05210         380,986.18214   
9                  Total      1,293,810.40899       2,356,955.54350   

   Head Room for Off-Air  On Air Days  Off-Air Days     Saturation Point  \
0          351,115.72055          170            74          4,744.80703   
1          161,862.54883          170            74          2,187.33174   
2          506,400.19031            4           240          2,110.00079   
3          113,499.09302          101           143            793.69995   
4          518,163.90404          129           115          4,505.77308   
5          254,847.98520            0           244          1,044.45896   
6                0.00000          244             0          1,382.92665   
7          100,302.36251          220            24          4,179.26510   
8          120,207.35439          194            50          2,404.14709   
9        2,126,399.15885         1232           964         23,352.41040   

      Non-Zero Average      Total Head Room  Total Head Room Ratio  
0          2,288.09561        768,756.66322                1.97636  
1            966.46071        369,410.62349                2.24841  
2         29,814.42000        506,400.19031                5.66169  
3            178.07673        175,677.03809                9.76757  
4          1,186.48600        946,351.93770                6.18302  
5                  nan                  nan                    nan  
6          1,000.00000         93,434.10362                0.38293  
7            692.99119        867,282.62419                5.74085  
8            440.30079        501,193.53653                5.89792  
9         36,566.83102      4,228,506.71716               37.85875

Bound Analysis result saved to: /Users/vn51u3a/Desktop/ms_new/Modeling/OLS Model/auto folder_structure/models/manual/sales_unit/mix_media_metrics/2021-11-09_model_outputs_v1/optimization_results/optimization_outputs_v55/bound_analysis.csv



Minimum              Average  \
Variable                                                          
M_ON_DIS_AT_SPEND                  0.12721              0.69672   
M_ON_DIS_CT_ROS_SPEND              0.08829              0.69672   
M_ON_DIS_HPLO_SPEND                1.17006              0.01639   
M_ON_DIS_KW_SPEND                  0.07958              0.41393   
M_OFF_DIS_WN_SPEND                 0.12928              0.52869   
M_OFF_DIS_PIN_SPEND                    nan                  nan   
M_NATIONAL_TV_SPEND                1.00000              1.00000   
M_SEARCH_AUTO_SPEND                0.07875              0.90164   
M_SEARCH_MAN_SPEND                 0.18720              0.79508   

                          Non-Zero Average      Threshold Point  \
Variable                                                          
M_ON_DIS_AT_SPEND                  1.00000              0.92530   
M_ON_DIS_CT_ROS_SPEND              1.00000              1.75087   
M_ON_DIS_HPLO_SPEND                1.33333              0.00000   
M_ON_DIS_KW_SPEND                  1.00000              3.02460   
M_OFF_DIS_WN_SPEND                 1.00000              2.91614   
M_OFF_DIS_PIN_SPEND                    nan                  nan   
M_NATIONAL_TV_SPEND                1.00000              0.61707   
M_SEARCH_AUTO_SPEND                1.00917              3.23167   
M_SEARCH_MAN_SPEND                 1.00518              3.03936   

                          Inflection Point     Saturation Point  \
Variable                                                          
M_ON_DIS_AT_SPEND                  1.49950              2.07369   
M_ON_DIS_CT_ROS_SPEND              2.00705              2.26324   
M_ON_DIS_HPLO_SPEND                0.01686              0.09436   
M_ON_DIS_KW_SPEND                  3.74083              4.45707   
M_OFF_DIS_WN_SPEND                 3.35686              3.79758   
M_OFF_DIS_PIN_SPEND                    nan                  nan   
M_NATIONAL_TV_SPEND                1.00000              1.38293   
M_SEARCH_AUTO_SPEND                4.65888              6.08609   
M_SEARCH_MAN_SPEND                 4.26395              5.48853   

                                   Maximum  
Variable                                    
M_ON_DIS_AT_SPEND                  5.50534  
M_ON_DIS_CT_ROS_SPEND              7.45291  
M_ON_DIS_HPLO_SPEND                1.45148  
M_ON_DIS_KW_SPEND                  7.14913  
M_OFF_DIS_WN_SPEND                 2.21569  
M_OFF_DIS_PIN_SPEND                    nan  
M_NATIONAL_TV_SPEND                1.00000  
M_SEARCH_AUTO_SPEND                4.27674  
M_SEARCH_MAN_SPEND                 2.26331